In [29]:
import pandas as pd
import numpy as np

driver_history = pd.read_parquet('../output_test/driver_histories_parquet')
driver_history = driver_history.reset_index(drop = True)
driver_history

,start_time,end_time,start_zone,end_zone,is_moving,has_passenger,driver_id
0,0.000000,57.210880,3,3,False,False,0
1,57.210880,66.343109,3,174,True,True,0
2,66.343109,79.271963,174,174,False,False,0
3,79.271963,100.798268,174,247,True,True,0
4,100.798268,157.099932,247,247,False,False,0
...,...,...,...,...,...,...,...
1368943,1490.061709,1515.547886,145,166,True,True,9999
1368944,1515.547886,1515.547886,166,166,False,False,9999
1368945,1515.547886,1543.881220,166,28,True,False,9999
1368946,1543.881220,1543.881220,28,28,False,False,9999


In [30]:
first_driver = driver_history.groupby('driver_id').get_group(0)

In [31]:
from shapely.geometry import Polygon, Point
from tqdm.notebook import tqdm
from joblib import dump, load

polygons, zone_dict = load('polygon_info')

In [32]:
from matplotlib.path import Path

#first get the extents of every zone
extent_dict = {k:(v.get_extents().min, v.get_extents().max) for k,v in zone_dict.items()}
extent_dict[1]

(array([141., 396.]), array([221., 467.]))

In [36]:
need_positions_generated = driver_history[(driver_history.start_time == 0) | (driver_history.is_moving)]
need_positions_generated

,start_time,end_time,start_zone,end_zone,is_moving,has_passenger,driver_id
0,0.000000,57.210880,3,3,False,False,0
1,57.210880,66.343109,3,174,True,True,0
3,79.271963,100.798268,174,247,True,True,0
5,157.099932,174.146099,247,41,True,True,0
7,313.912293,322.120921,41,74,True,False,0
...,...,...,...,...,...,...,...
1368939,1433.155557,1475.712826,132,230,True,True,9999
1368941,1475.712826,1490.061709,230,145,True,False,9999
1368943,1490.061709,1515.547886,145,166,True,True,9999
1368945,1515.547886,1543.881220,166,28,True,False,9999


In [37]:
#for each zone, how many points need to be sampled
points_required = need_positions_generated.groupby('end_zone').start_time.count()
points_required

end_zone
1      2504
2         3
3      1602
4      1940
5       236
       ... 
259    1601
260    3323
261    2458
262    2559
263    3551
Name: start_time, Length: 262, dtype: int64

In [78]:
def generate_points(zone_id, num_required, zone_dict = zone_dict, extent_dict = extent_dict):
    bounds = extent_dict[zone_id]
    path = zone_dict[zone_id]
    generated_points = np.array([])
    while len(generated_points) < num_required:
        xs = np.random.uniform(bounds[0][0], bounds[1][0], size = num_required)
        ys = np.random.uniform(bounds[0][1], bounds[1][1], size = num_required)
        points = np.c_[xs,ys]
        if len(generated_points) == 0:
            generated_points = points
        else:
            generated_points = np.append(generated_points, points, axis = 0)
    return generated_points[:num_required].round(2)

In [79]:
g = []
for i in tqdm(points_required.index):
    positions_generated = generate_points(i, points_required.loc[i])
    pos_df = pd.DataFrame(positions_generated, columns = ['x','y'])
    pos_df['end_zone'] = i
    g.append(pos_df)
g = pd.concat(g)
g

  0%|          | 0/262 [00:00<?, ?it/s]

,x,y,end_zone
0,193.59,436.10,1
1,176.11,411.72,1
2,145.35,434.34,1
3,207.60,445.88,1
4,168.61,442.01,1
...,...,...,...
3546,663.38,257.41,263
3547,659.27,266.13,263
3548,651.68,257.56,263
3549,645.11,250.02,263


In [80]:
#need to assign these points to the drivers
position_dfs = []
for group in tqdm(need_positions_generated.groupby(['end_zone']), position = 0, leave = True):
    ind = group[1].index
    zone = group[1].end_zone.iloc[0]
    
    #create a new dataframe with the index and corresponding values from generated points
    position_df = pd.DataFrame(g[g.end_zone == zone][['x','y']].values, index = ind, columns = ['x','y'])
    position_dfs.append(position_df)
position_dfs = pd.concat(position_dfs).sort_index()
position_dfs

  0%|          | 0/262 [00:00<?, ?it/s]

,x,y
0,887.40,100.71
1,817.85,62.34
3,714.43,161.47
5,675.40,204.80
7,704.02,226.41
...,...,...
1368939,587.60,306.00
1368941,653.93,331.49
1368943,639.35,195.34
1368945,932.57,404.22


In [81]:
driver_history[['end_x','end_y']] = position_dfs
driver_history

,start_time,end_time,start_zone,end_zone,is_moving,has_passenger,driver_id,end_x,end_y
0,0.000000,57.210880,3,3,False,False,0,887.40,100.71
1,57.210880,66.343109,3,174,True,True,0,817.85,62.34
2,66.343109,79.271963,174,174,False,False,0,NaN,NaN
3,79.271963,100.798268,174,247,True,True,0,714.43,161.47
4,100.798268,157.099932,247,247,False,False,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1368943,1490.061709,1515.547886,145,166,True,True,9999,639.35,195.34
1368944,1515.547886,1515.547886,166,166,False,False,9999,NaN,NaN
1368945,1515.547886,1543.881220,166,28,True,False,9999,932.57,404.22
1368946,1543.881220,1543.881220,28,28,False,False,9999,NaN,NaN


In [91]:
#assuming the end zones are alternating NaN and non-null values
#for every zone but the first set the null values to the non-null value above it

start_end_dfs = []
#the start x,y equals the end x,y but shifted up 1 (except for the first, which is the same as the end)
for did, group in tqdm(driver_history.groupby('driver_id'), position = 0, leave = True):
    first_move_end = group.iloc[0][['end_x','end_y']].values
    next_moves_end = group.iloc[1:][['end_x','end_y']].values
    next_moves_end[1::2] = next_moves_end[:-1:2]
    end_moves = np.r_[[first_move_end], next_moves_end]
    start_moves = np.r_[[first_move_end],end_moves[:-1]]
    sedf = pd.DataFrame(np.c_[start_moves, end_moves], 
                        index = group.index, 
                        columns = ['startx','starty','endx','endy'])
    start_end_dfs.append(sedf)
start_end_dfs = pd.concat(start_end_dfs)
start_end_dfs

  0%|          | 0/10000 [00:00<?, ?it/s]

,startx,starty,endx,endy
0,887.4,100.71,887.4,100.71
1,887.4,100.71,817.85,62.34
2,817.85,62.34,817.85,62.34
3,817.85,62.34,714.43,161.47
4,714.43,161.47,714.43,161.47
...,...,...,...,...
1368943,653.93,331.49,639.35,195.34
1368944,639.35,195.34,639.35,195.34
1368945,639.35,195.34,932.57,404.22
1368946,932.57,404.22,932.57,404.22


In [92]:
driver_history[['startx','starty','endx','endy']] = start_end_dfs
driver_history.drop(columns = ['end_x','end_y'],inplace = True)

In [97]:
#make an animation object for each driver
#need to calculate velocity and frame data
driver_history

,start_time,end_time,start_zone,end_zone,is_moving,has_passenger,driver_id,startx,starty,endx,endy
0,0.000000,57.210880,3,3,False,False,0,887.4,100.71,887.4,100.71
1,57.210880,66.343109,3,174,True,True,0,887.4,100.71,817.85,62.34
2,66.343109,79.271963,174,174,False,False,0,817.85,62.34,817.85,62.34
3,79.271963,100.798268,174,247,True,True,0,817.85,62.34,714.43,161.47
4,100.798268,157.099932,247,247,False,False,0,714.43,161.47,714.43,161.47
...,...,...,...,...,...,...,...,...,...,...,...
1368943,1490.061709,1515.547886,145,166,True,True,9999,653.93,331.49,639.35,195.34
1368944,1515.547886,1515.547886,166,166,False,False,9999,639.35,195.34,639.35,195.34
1368945,1515.547886,1543.881220,166,28,True,False,9999,639.35,195.34,932.57,404.22
1368946,1543.881220,1543.881220,28,28,False,False,9999,932.57,404.22,932.57,404.22


In [141]:
def vf(df, st, et, sx, sy, ex, ey, scaling=30, fps=30):
    frames = (df[et] - df[st]) / scaling * fps
    frames[frames == 0] = 0.0001
    frames = frames.values
    direction = (df[[ex, ey]].values - df[[sx, sy]].values).astype('float')
    velocity = (1/f * direction.T).T
    
    return velocity, frames
v, f = vf(driver_history, 'start_time', 'end_time', 'startx','starty','endx','endy')

In [144]:
driver_history[['vx','vy']] = v
driver_history['frames'] = f

In [145]:
driver_history

,start_time,end_time,start_zone,end_zone,is_moving,has_passenger,driver_id,startx,starty,endx,endy,vx,vy,frames
0,0.000000,57.210880,3,3,False,False,0,887.4,100.71,887.4,100.71,0.000000,0.000000,57.210880
1,57.210880,66.343109,3,174,True,True,0,887.4,100.71,817.85,62.34,-7.615884,-4.201603,9.132229
2,66.343109,79.271963,174,174,False,False,0,817.85,62.34,817.85,62.34,0.000000,0.000000,12.928854
3,79.271963,100.798268,174,247,True,True,0,817.85,62.34,714.43,161.47,-4.804354,4.605063,21.526305
4,100.798268,157.099932,247,247,False,False,0,714.43,161.47,714.43,161.47,0.000000,0.000000,56.301663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1368943,1490.061709,1515.547886,145,166,True,True,9999,653.93,331.49,639.35,195.34,-0.572075,-5.342111,25.486178
1368944,1515.547886,1515.547886,166,166,False,False,9999,639.35,195.34,639.35,195.34,0.000000,0.000000,0.000100
1368945,1515.547886,1543.881220,166,28,True,False,9999,639.35,195.34,932.57,404.22,10.348941,7.372235,28.333333
1368946,1543.881220,1543.881220,28,28,False,False,9999,932.57,404.22,932.57,404.22,0.000000,0.000000,0.000100


In [1]:
#reformulate a driver's animation to just store the matrix above
class DriverAnimation:
    
    sx, sy, ex, ey = 7, 8, 9, 10
    st, et, vx, vy = 0, 1, 11, 12
    is_move, has_pass, f = 4,5,13
    
    def __init__(self, movement_matrix):
        self.movements = movement_matrix
        self.center = movement_matrix[0][self.sx], movement_matrix[0][self.sy]
        self.current_movement_index = 0
        self.frame_count = 0
        
    def update(self):
        if self.current_movement_index < self.movements.shape[0]:
            m = self.movements[self.current_movement_index]
            if self.frame_count == 0:
                self.center = m[self.sx], m[self.sy]
            elif self.frame_count > m[self.f]:
                self.center = m[self.ex], m[self.ey]
                self.current_movement_index += 1
                self.frame_count = 0
            else:
                self.center[0] += m[self.vx]
                self.center[1] += m[self.vy]
                self.frame_count += 1
        return self.center
    
    #0 for not moving
    #1 for moving with passenger
    #2 for moving without passenger
    def state(self):
        if self.current_movement_index >= self.movements.shape[0]:
            return 0
        else:
            m = self.movements[self.current_movement_index]
            if m[self.has_pass]:
                return 1
            elif m[self.is_move]:
                return 2
            else:
                return 0
    
    def get_time(self):
        if self.current_movement_index >= self.movements.shape[0]:
            return None
        else:
            m = self.movements[self.current_movement_index]
            return m[self.st] + (m[self.st] - m[self.et])/frames * self.frame_count
            